In [85]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly

In [86]:
# Price index weights on different goods
# https://ec.europa.eu/eurostat/databrowser/view/PRC_HICP_INW__custom_2846029/default/table?lang=en
weights = pd.read_csv('data/prc_hicp_inw__custom_2846029_page_tabular.tsv', sep="\t")


# Clean weights
def extract_coicop(col, position):
    return col.str.split(",", expand=True)[position]

weights["coicop"] = extract_coicop(weights["freq,coicop,geo\TIME_PERIOD"], 1)
weights.columns = [c.strip() for c in weights.columns]
weights = weights.drop("freq,coicop,geo\TIME_PERIOD", axis=1)
weights.head()

,2019,2020,2021,2022,coicop
0,90.20,82.93,94.82,121.47,AP
1,3.85,3.63,4.21,4.56,APF
2,86.36,79.30,90.60,116.92,APM
3,30.45,26.48,30.47,31.62,AP_NNRG
4,59.76,56.45,64.35,89.85,AP_NRG


In [87]:
# Monthly inflation data
# https://ec.europa.eu/eurostat/databrowser/view/PRC_HICP_MMOR__custom_2846191/default/table?lang=en
monthly_inflation = pd.read_csv('data/prc_hicp_mmor__custom_2846191_page_tabular.tsv', sep="\t")

# Clean monthly inflation data
monthly_inflation["coicop"] = extract_coicop(monthly_inflation["freq,unit,coicop,geo\TIME_PERIOD"], 2)
monthly_inflation.columns = [c.strip() for c in monthly_inflation.columns]
monthly_inflation = monthly_inflation.drop("freq,unit,coicop,geo\TIME_PERIOD", axis=1)
monthly_inflation.head()

,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,coicop
0,0.5,10.7,-7.2,7.8,29.2,-10.7,-5.0,3.0,24.3,:,AP
1,0.0,0.0,0.0,0.0,0.0,1.3,0.0,0.0,0.0,:,APF
2,0.5,11.2,-7.5,8.2,30.3,-11.1,-5.2,3.2,25.4,:,APM
3,0.0,0.0,0.0,0.0,0.1,0.4,0.3,2.1,0.7,:,AP_NNRG
4,0.7,15.0,-9.7,10.9,39.3,-14.6,-7.2,3.4,34.8,:,AP_NRG


In [88]:
# COICOP classifier
# https://ec.europa.eu/eurostat/ramon/nomenclatures/index.cfm?TargetUrl=LST_CLS_DLD&StrLanguageCode=EN&StrNom=CL_COICOP&StrLayoutCode=LINEAR
coicop = pd.read_excel('data/COICOP.xls', skiprows=2)

# Clean COICOP
coicop.columns = [c.strip() for c in coicop.columns]
coicop = coicop.drop(labels=["DE_DESC", "FR_DESC", "DESC_AGGR", "INT_CODE"], axis=1)
coicop.head()

,CODE_DIFF,LEVEL,EN_DESC
0,TOTAL,1,Total
1,TOT_X_CP041_042,1,Total except actual rents
2,CP00,1,All-items HICP
3,CP01,1,Food and non-alcoholic beverages
4,CP011,2,Food


# Treemap

In [89]:
MAX_LEVEL = 4 # How deep detail do we want to show?

merged = monthly_inflation.merge(coicop, left_on="coicop", right_on="CODE_DIFF")
merged = merged[merged["coicop"].str.slice(0, 2) == "CP"]
merged = merged[merged["LEVEL"] == MAX_LEVEL]
merged = merged.reset_index()

# Add hierarchical description
def get_level_description(codes, level=1):
    """Get the COICOP hierarchical description corresponding to `level` in `code`."""
    level_code = codes.str.slice(0, 3+level)
    df = pd.DataFrame({"level_code": level_code})
    df = df.merge(coicop, left_on="level_code", right_on="CODE_DIFF")
    level_coicop_code = df["CODE_DIFF"]
    level_coicop_desc = df["EN_DESC"]
    return level_coicop_desc

for level in range(1, MAX_LEVEL+1):
    merged[f"L{level}_DESC"] = get_level_description(merged["coicop"], level=level)
    
# Multiply percentage growths by index weight
def get_coicop_weight(codes, month: str):
    """Get the weights in price index, given the COICOP code."""
    year = month[0:4]
    df = pd.DataFrame({"coicop": codes})
    df = df.merge(weights, left_on="coicop", right_on="coicop")
#    df = df.rename({year: "weight"})
#    df = df["coicop", "weight"]
    
    return df[year]

In [90]:
def get_adjacent_month(month, delta):
    months = merged.columns[1:10].tolist()
    current_idx = months.index(month)
    if current_idx < 0 or current_idx + delta >= len(months):
        return None
    return months[current_idx + delta]

def plot_month(month, show=True):
    percs = merged[month]
    percs_capped = np.maximum(np.zeros(percs.shape), percs.to_numpy())
    weights = get_coicop_weight(merged["coicop"], month=month)
    scale = 0.01 * 1000 / weights.sum() # 0.01 is from percent to 0...1 conversion; 1000 is to scale to 1000EUR
    vals = percs_capped * weights * scale
    columns = {
        "val": vals,
        "perc": percs
    }
    
    for level in range(1, MAX_LEVEL+1):
        columns[f"L{level}_DESC"] = merged[f"L{level}_DESC"]
    df = pd.DataFrame(columns)
    
    TREEMAP_LEVELS = [f"L{level}_DESC" for level in range(1, MAX_LEVEL+1)]
    
    # Title and links
    perc_growth = monthly_inflation[monthly_inflation["coicop"] == "CP00"][month].iloc[0]
    prev_month = get_adjacent_month(month, -1)
    prev_month_link = f'<a href="https://taivoai-public.s3.eu-central-1.amazonaws.com/inflation_charts/{prev_month}.html">{prev_month}</a>'
    next_month = get_adjacent_month(month, 1)
    next_month_link = f'<a href="https://taivoai-public.s3.eu-central-1.amazonaws.com/inflation_charts/{next_month}.html">{next_month}</a>'
    
    fig_title = f"""
<b>Consumer price growth by spending category, {month} ({perc_growth}% month total) </b><br>
{"Prev: " + prev_month_link if prev_month is not None else ""}
{"Next: " + next_month_link if next_month is not None else ""}
| Data: <a href="https://ec.europa.eu/eurostat/databrowser/view/PRC_HICP_MMOR__custom_2846191/default/table?lang=en">Eurostat</a>
| Author: <a href="https://taivo.ai/estonia-inflation-2022">Taivo Pungas</a>
    """
    
    
    fig = px.treemap(df, path=[px.Constant("all"), *TREEMAP_LEVELS], values="val",
                    title=fig_title)
    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin = dict(t=100, l=25, r=25, b=25), font=dict(size=13))
    
    plotly.io.write_html(fig, f"figures/{month}.html")
    
    if show:
        fig.show()

    
plot_month("2022-02")

In [91]:
for month in merged.columns[1:10]:
    print(month)
    plot_month(month, show=False)

2021-08
2021-09
2021-10
2021-11
2021-12
2022-01
2022-02
2022-03
2022-04


# Experimenting with other kinds of visualisation

In [92]:
monthly_bars = monthly_inflation.merge(coicop, left_on="coicop", right_on="CODE_DIFF")
monthly_bars = monthly_bars[monthly_bars["coicop"].str.slice(0, 2) == "CP"]
monthly_bars = monthly_bars[monthly_bars["coicop"].str.slice(0, 4) != "CP00"]
monthly_bars = monthly_bars[monthly_bars["LEVEL"] == 1]

costs = dict()
initial_costs = monthly_bars.merge(weights, left_on="coicop", right_on="coicop").sort_values("coicop")["2021"].to_numpy()
costs["EN_DESC"] = monthly_bars.merge(weights, left_on="coicop", right_on="coicop").sort_values("coicop")["EN_DESC"]
prev_month = None
for month in monthly_bars.columns[:9]:
    if month == monthly_bars.columns[0]:
        costs[month] = initial_costs
    else:
        costs[month] = costs[prev_month] * (1.0 + monthly_bars.sort_values("coicop")[month].to_numpy() / 100)
    prev_month = month
    
cost_evolution = pd.DataFrame(costs)
cost_evolution

,EN_DESC,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04
0,Food and non-alcoholic beverages,220.74,219.63630,217.220301,221.564707,221.121577,229.745319,238.245896,241.581338,247.137709
1,"Alcoholic beverages, tobacco and narcotics",67.00,67.13400,67.939608,67.939608,67.464031,68.138671,68.888196,69.094861,69.233051
2,Clothing and footwear,56.00,58.91200,60.266976,61.351782,60.615560,56.918011,57.031847,58.856866,60.034003
3,"Housing, water, electricity, gas and other fuels",142.45,153.13375,146.702132,154.183941,185.791649,172.972025,167.436921,173.297213,208.823141
4,"Furnishings, household equipment and routine h...",68.34,68.95506,68.817150,69.367687,70.200099,69.919299,71.317685,72.244815,73.328487
5,Health,60.87,60.87000,61.235220,61.418926,61.971696,63.025215,63.214290,63.530362,63.657423
6,Transport,130.95,131.08095,135.799864,138.244262,138.244262,139.903193,144.799805,157.397388,156.138209
7,Communications,48.87,48.57678,48.673934,48.722607,47.650710,47.317155,47.222521,47.694746,47.599357
8,Recreation and culture,77.64,77.87292,78.028666,78.730924,78.809655,78.494416,79.828821,80.387623,82.075763
9,Education,10.44,10.56528,10.565280,10.607541,10.607541,10.649971,10.671271,10.671271,10.681942


In [93]:
cost_evolution_long = cost_evolution.melt(id_vars=["EN_DESC"], 
        var_name="month", 
        value_name="eur_budget")
cost_evolution_long

,EN_DESC,month,eur_budget
0,Food and non-alcoholic beverages,2021-08,220.740000
1,"Alcoholic beverages, tobacco and narcotics",2021-08,67.000000
2,Clothing and footwear,2021-08,56.000000
3,"Housing, water, electricity, gas and other fuels",2021-08,142.450000
4,"Furnishings, household equipment and routine h...",2021-08,68.340000
...,...,...,...
103,Communications,2022-04,47.599357
104,Recreation and culture,2022-04,82.075763
105,Education,2022-04,10.681942
106,Restaurants and hotels,2022-04,59.226567


In [94]:
fig = px.bar(cost_evolution_long, x="EN_DESC", y="eur_budget", color="EN_DESC",
  animation_frame="month", range_y=[0,250])
fig.update_xaxes(visible=False, showticklabels=False)
fig.update_layout(
    xaxis_title="Spending category",
    yaxis_title="Budget",
    legend_title="Spending category",
)
fig.show()

In [95]:
monthly_long = monthly_bars.drop(["coicop", "CODE_DIFF", "LEVEL", "2022-05"], axis=1)
monthly_long = monthly_long.melt(id_vars=["EN_DESC"], 
        var_name="month", 
        value_name="perc_increase")
monthly_long

,EN_DESC,month,perc_increase
0,Food and non-alcoholic beverages,2021-08,0.7
1,"Alcoholic beverages, tobacco and narcotics",2021-08,0.6
2,Clothing and footwear,2021-08,0.9
3,"Housing, water, electricity, gas and other fuels",2021-08,0.9
4,"Furnishings, household equipment and routine h...",2021-08,1.3
...,...,...,...
103,Communications,2022-04,-0.2
104,Recreation and culture,2022-04,2.1
105,Education,2022-04,0.1
106,Restaurants and hotels,2022-04,4.2


In [96]:
fig = px.bar(monthly_long, x="EN_DESC", y="perc_increase", color="EN_DESC",
  animation_frame="month", range_y=[-20,20])
fig.show()